In [ ]:
# | export

import foodcosts.core as core
import matplotlib.pyplot as plt
import pandas as pd
import math
import smtplib
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from IPython.display import display_markdown
import markdown


In [ ]:
# | export

days_ago = 0


def get_today():
    return core.get_previous_n_day(days_ago)


def get_tomorrow():
    return core.get_previous_n_day(-1 + days_ago)


def get_weekday(date):
    date_object = datetime.strptime(date, '%Y-%m-%d')
    return date_object.strftime('%A')


def get_last_four_weekday_dates():
    last_four_weekday_dates = [core.get_previous_n_day(
        i+days_ago) for i in range(7, 29, 7)]
    return last_four_weekday_dates


In [ ]:
# | export

def get_email_subject(date):
    weekday = get_weekday(date)
    return f"""{date}'s - {weekday} -report"""


def get_daily_report_top_md(date, day, total_sales, shops_total_sales, re_restaurants_total_sales, events_total_sales):
    total_sales = int(total_sales)
    shops_total_sales = int(shops_total_sales)
    re_restaurants_total_sales = int(re_restaurants_total_sales)
    events_total_sales = int(events_total_sales)

    return f"""
# {date}'s - {day} -report

  

Today sales were {total_sales} EUR across the business:

  

- {shops_total_sales} EUR sold in stores

- {re_restaurants_total_sales} EUR sold in corporate restaurants

- {events_total_sales} EUR sold in events

"""


def get_store_sales_title_md():
    return f"""
## Store sales

### Stores sales per shop


"""


def get_sales_details_md(name, total_sale, weekday, avg_sales, change_pct, change):
    total_sale = int(total_sale)
    avg_sales = int(avg_sales)
    change_pct = round(change_pct, 1)

    if(change_pct < 10):
        return f"""
- {name} sold {total_sale} EUR. Average of last 4 {weekday}’s was {avg_sales} EUR. That’s {change_pct}% {change} vs. average of last 4 {weekday}'s

"""
    else:
        return f"""
- {name} sold {total_sale} EUR. Average of last 4 {weekday}’s was {avg_sales} EUR. That’s **{change_pct}%** {change} vs. average of last 4 {weekday}'s

"""


def get_product_stock_out_title_md(outlet_type):
    return f"""
### Products that were likely in stock out

The products that were sold in the each days of last 7 days for a given {outlet_type} but didn’t sell today:
 
"""


def get_stock_out_product_details_md(product_name, outlet):
    return f"""
- {product_name} in {outlet}

"""


def get_corporate_restaurant_sales_title_md():
    return f"""
## Corporate restaurant sales

### Sales per corporate restaurant

"""


def get_no_sales_in_corporate_restaurant_md():

    return f"""
There is no sales in the corporate restaurant today

"""


def get_cus_at_risk_of_loosing_title_md():
    return f"""
### Repeat customers we are at risk of loosing

Customers who didn’t buy in the last 7 days but who bought at least once a week for the last 10 weeks before that:
"""


def get_at_risk_of_loosing_title_cus_details_md(customer_name, total_purchase, location):
    total_purchase = int(total_purchase)
    return f"""
- {customer_name}. Bought {total_purchase} EUR in the last 10 weeks. Buys the most in {location}

"""


In [ ]:
# | export

def get_sales_details_html(sales_details_md, change):
    sales_details_html = markdown.markdown(sales_details_md)
    if (change == "increase"):
        return sales_details_html[:9] + f"""<span style="background-color:#00FF00">""" + sales_details_html[9:-11] + f"""</span>""" + sales_details_html[-11:]
    elif (change == "decrease"):
        return sales_details_html[:9] + f"""<span style="background-color:#58111A; color: #FFFFFF">""" + sales_details_html[9:-11] + f"""</span>""" + sales_details_html[-11:]
    else:
        return sales_details_html


In [ ]:
# | export
def get_corporate_restaurant_names():
    return [name for (i, name) in core.make_sql_query(
        "select * from v_resto_entreprise")]


def get_all_shop_sales_as_pd(dates):
    dates_str = ", ".join("'" + str(val) + "'" for val in dates)

    query = f"""
        SELECT
            v_shop.id as id,
            v_shop.name as name,
            day,
            total
        FROM v_shop_sales_statistics
        LEFT JOIN
            v_shop on v_shop.id = v_shop_sales_statistics.shop_id
        WHERE
            v_shop.name IS NOT NULL AND
        day IN ({dates_str});
        """

    return core.get_query_res_as_pd(query)


def get_all_corporate_restaurant_sales_as_pd(dates):
    dates_str = ", ".join("'" + str(val) + "'" for val in dates)

    query = f"""
        SELECT 
            v_resto_entreprise.id as id,
            v_resto_entreprise.name as name,
            day,
            total
        FROM v_shop_sales_statistics
        LEFT JOIN 
            v_resto_entreprise on v_resto_entreprise.id = v_shop_sales_statistics.shop_id 
        WHERE 
        v_resto_entreprise.name IS NOT NULL AND     
        day IN ({dates_str});
        """

    return core.get_query_res_as_pd(query)


def get_all_event_sales_as_pd(start_date, end_date):
    query = f"""
        SELECT COALESCE(SUM(amount_total), 0) as total
        FROM sale_order 
        WHERE 
        state = 'sale' AND date_order BETWEEN '{start_date}' and '{end_date}';
        """

    return core.get_query_res_as_pd(query)


In [ ]:
# | export

def get_products_sold_for_shops_as_pd(start_date, end_date):
    query = f"""
        SELECT v_shop.name AS shop_name, 
               DATE(v_pos_order.created_at) AS day, 
               v_product.name AS product_name, 
               SUM(v_pos_order_line.quantity) AS units_sold, 
               SUM(v_pos_order_line.quantity * v_product.price) AS total_sales
        FROM v_pos_order_line
        LEFT JOIN v_product ON v_product.id = v_pos_order_line.product_id
        LEFT JOIN v_pos_order ON v_pos_order.id = v_pos_order_line.order_id
        LEFT JOIN v_pos_session ON v_pos_session.id = v_pos_order.session_id
        LEFT JOIN v_pos ON v_pos.id = v_pos_session.pos_id
        LEFT JOIN v_shop ON v_shop.id = v_pos.shop_id
        WHERE v_pos_order.created_at BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY v_shop.id, v_shop.name, day, v_product.id, v_product.name
        ORDER BY v_shop.name, day ASC;
        """

    return core.get_query_res_as_pd(query)


def get_products_sold_for_re_as_pd(start_date, end_date):
    query = f"""
        SELECT v_resto_entreprise.name AS shop_name, 
               DATE(v_pos_order.created_at) AS day, 
               v_product.name AS product_name, 
               SUM(v_pos_order_line.quantity) AS units_sold, 
               SUM(v_pos_order_line.quantity * v_product.price) AS total_sales
        FROM v_pos_order_line
        LEFT JOIN v_product ON v_product.id = v_pos_order_line.product_id
        LEFT JOIN v_pos_order ON v_pos_order.id = v_pos_order_line.order_id
        LEFT JOIN v_pos_session ON v_pos_session.id = v_pos_order.session_id
        LEFT JOIN v_pos ON v_pos.id = v_pos_session.pos_id
        LEFT JOIN v_resto_entreprise ON v_resto_entreprise.id = v_pos.shop_id
        WHERE v_pos_order.created_at BETWEEN '{start_date}' AND '{end_date}'
        GROUP BY v_resto_entreprise.id, v_resto_entreprise.name, day, v_product.id, v_product.name
        ORDER BY v_resto_entreprise.name, day ASC;
    """
    return core.get_query_res_as_pd(query)


In [ ]:
# | export

def get_pos_order(start_date, end_date):
    query = f"""
        SELECT
            DATE(pos_order.create_date) AS day,
            pos_order.partner_id AS customer_id ,
            pos_order.amount_total AS amount_total,
            stock_warehouse.name  AS location, 
            v_customer.name AS customer_name
        FROM pos_order
            LEFT JOIN v_customer ON v_customer.id = pos_order.partner_id
            LEFT JOIN stock_warehouse ON stock_warehouse.id = pos_order.location_id
        WHERE pos_order.create_date BETWEEN '{start_date}' AND '{end_date}'
            AND pos_order.partner_id IS NOT NULL
        GROUP BY day, customer_id, amount_total, location_id, v_customer.name, stock_warehouse.name
        ORDER BY day;
        """

    return core.get_query_res_as_pd(query)

def get_pos_order_as_pd(start_date, end_date):
    query = f"""
        SELECT
	        DATE(pos_order.create_date) AS day,
	        pos_order.partner_id AS customer_id,
	        pos_order.amount_total AS amount_total,
	        vc.name AS customer_name,
	        sw.name as location
        FROM pos_order
	        LEFT JOIN v_customer AS vc ON vc.id = pos_order.partner_id
	        LEFT JOIN v_pos_session AS vs ON pos_order.session_id = vs.id
	        LEFT JOIN v_pos AS vpos ON vpos.id = vs.pos_id
	        LEFT JOIN stock_warehouse AS sw ON sw.id = vpos.shop_id
        WHERE 
	        pos_order.create_date BETWEEN '{start_date}' AND '{end_date}'	
	        AND	pos_order.state::text = ANY (ARRAY['done'::character varying::text, 'invoiced'::character varying::text, 'paid'::character varying::text]);
        """

    return core.get_query_res_as_pd(query)


In [ ]:
# | export

def compare_value_in_percentage(x, y):
    try:
        return round((x*100/(y)), 2)
    except ZeroDivisionError:
        return float('NaN')


def get_change_percentage(x, y):
    change_percentage = compare_value_in_percentage(x, y)
    change = "decrease" if change_percentage < 0 else "increase"
    abs_change_percentage = abs(change_percentage)

    return change, abs_change_percentage


In [ ]:
# | export

def get_daily_report_top():
    today = get_today()
    tomorrow = get_tomorrow()

    all_shop_sales = get_all_shop_sales_as_pd([today])
    all_corporate_restaurant_sales = get_all_corporate_restaurant_sales_as_pd([
        today])
    all_event_sales = get_all_event_sales_as_pd(today, tomorrow)

    weekday = get_weekday(today)

    total_sales_at_shops = all_shop_sales['total'].values.sum()
    total_sales_at_corporate_restaurants = all_corporate_restaurant_sales['total'].values.sum(
    )
    total_sales_at_events = all_event_sales['total'].values.sum()

    total_sales = total_sales_at_shops + \
        total_sales_at_corporate_restaurants + total_sales_at_events

    daily_report_top_md = get_daily_report_top_md(
        today,
        weekday,
        total_sales,
        total_sales_at_shops,
        total_sales_at_corporate_restaurants,
        total_sales_at_events)

    daily_report_top_html = markdown.markdown(daily_report_top_md)

    return daily_report_top_md, daily_report_top_html


In [ ]:
# | export

def get_store_sales_title():
    store_sales_title_md = get_store_sales_title_md()
    store_sales_title_html = markdown.markdown(store_sales_title_md)

    return store_sales_title_md, store_sales_title_html


def get_sales_per_shop():
    today = get_today()
    last_four_weekday_dates = get_last_four_weekday_dates()

    all_shop_sales_for_last_four_weekdays = get_all_shop_sales_as_pd(
        last_four_weekday_dates)

    all_shop_sales = get_all_shop_sales_as_pd([today])
    shop_list = core.get_shop_names()

    sales_per_shop_md = ""
    sales_per_shop_html = ""
    for shop in shop_list:
        sale_by_shop = all_shop_sales[all_shop_sales['name']
                                      == shop]['total'].values.sum()
        sale_by_shop_in_last_four_weeks_at_this_weekday = all_shop_sales_for_last_four_weekdays[
            all_shop_sales_for_last_four_weekdays['name'] == shop]['total'].values.sum()
        avg_sale_at_this_weekday_in_last_four_week = round(
            (sale_by_shop_in_last_four_weeks_at_this_weekday / 4), 2)

        difference_in_sale = sale_by_shop - avg_sale_at_this_weekday_in_last_four_week

        change, change_percentage = get_change_percentage(
            difference_in_sale, avg_sale_at_this_weekday_in_last_four_week)

        shop_sales_md = get_sales_details_md(shop, sale_by_shop, get_weekday(
            today), avg_sale_at_this_weekday_in_last_four_week, change_percentage, change)

        sales_per_shop_md += shop_sales_md
        sales_per_shop_html += get_sales_details_html(shop_sales_md, change)

    return sales_per_shop_md, sales_per_shop_html


In [ ]:
# | export

def get_shop_product_stock_out_title():
    shop_product_stock_out_title_md = get_product_stock_out_title_md("shop")
    shop_product_stock_out_title_html = markdown.markdown(
        shop_product_stock_out_title_md)

    return shop_product_stock_out_title_md, shop_product_stock_out_title_html


def get_shop_product_likely_to_stock_out():
    today = get_today()
    start_date = core.get_previous_n_day(7 + days_ago)
    end_date = core.get_previous_n_day(-1 + days_ago)

    shop_list = core.get_shop_names()

    product_sold_for_shops = {}
    products_sold_for_shops_pd = get_products_sold_for_shops_as_pd(
        start_date, end_date)

    for shop in shop_list:
        product_sold_for_shops[shop] = products_sold_for_shops_pd[products_sold_for_shops_pd['shop_name'] == shop]

    all_stock_out_product_md = ""

    for shop in shop_list:
        products_sold_for_shops = product_sold_for_shops[shop]
        sales_frequency = products_sold_for_shops['product_name'].value_counts(
        )

        for product, count in sales_frequency.items():
            if (count == 7):
                sales_today = products_sold_for_shops.loc[(products_sold_for_shops['product_name'] == product) & (
                    pd.to_datetime(products_sold_for_shops['day']) == pd.to_datetime(today))]
                if (len(sales_today) == 0):
                    all_stock_out_product_md += get_stock_out_product_details_md(
                        product, shop)

    all_stock_out_product_html = markdown.markdown(all_stock_out_product_md)

    return all_stock_out_product_md, all_stock_out_product_html


In [ ]:
# | export

def get_re_sales_title():
    re_sales_title_md = get_corporate_restaurant_sales_title_md()
    re_sales_title_html = markdown.markdown(re_sales_title_md)

    return re_sales_title_md, re_sales_title_html


def get_sales_per_re():
    today = get_today()
    all_re_sales_today = get_all_corporate_restaurant_sales_as_pd([today])

    total_sales_at_re = all_re_sales_today['total'].values.sum()

    if (total_sales_at_re == 0):
        no_sales_in_re_md = get_no_sales_in_corporate_restaurant_md()
        return no_sales_in_re_md, markdown.markdown(no_sales_in_re_md)

    last_four_weekday_dates = get_last_four_weekday_dates()

    all_re_sales_for_last_four_weekdays = get_all_corporate_restaurant_sales_as_pd(
        last_four_weekday_dates)

    corporate_restaurant_list = get_corporate_restaurant_names()

    all_re_sales_md = ""
    all_re_sales_html = ""
    for corporate_restaurant in corporate_restaurant_list:
        sales_by_re_today = all_re_sales_today[all_re_sales_today['name']
                                               == corporate_restaurant]['total'].values.sum()
        sales_by_re_in_last_four_weekdays = all_re_sales_for_last_four_weekdays[
            all_re_sales_for_last_four_weekdays['name'] == corporate_restaurant]['total'].values.sum()
        avg_sale_at_this_weekday_in_last_four_week = round(
            (sales_by_re_in_last_four_weekdays / 4), 2)

        difference_in_sale = sales_by_re_today - \
            avg_sale_at_this_weekday_in_last_four_week

        change, change_percentage = get_change_percentage(
            difference_in_sale, avg_sale_at_this_weekday_in_last_four_week)

        re_sales_md = get_sales_details_md(corporate_restaurant, sales_by_re_today, get_weekday(
            today), avg_sale_at_this_weekday_in_last_four_week, change_percentage, change)

        all_re_sales_md += re_sales_md
        all_re_sales_html += get_sales_details_html(re_sales_md, change)

    return all_re_sales_md, all_re_sales_html


In [ ]:
# | export

def get_re_product_stock_out_title():
    shop_product_stock_out_title_md = get_product_stock_out_title_md(
        "corporate restaurant")
    shop_product_stock_out_title_html = markdown.markdown(
        shop_product_stock_out_title_md)

    return shop_product_stock_out_title_md, shop_product_stock_out_title_html


def get_re_product_likely_to_stock_out():
    today = get_today()

    all_re_sales_today = get_all_corporate_restaurant_sales_as_pd([today])

    total_sales_at_re = all_re_sales_today['total'].values.sum()

    if (total_sales_at_re == 0):
        no_sales_in_re_md = get_no_sales_in_corporate_restaurant_md()
        return no_sales_in_re_md, markdown.markdown(no_sales_in_re_md)

    start_date = core.get_previous_n_day(30 + days_ago)
    end_date = core.get_previous_n_day(-1 + days_ago)

    dates_in_last_months = [core.get_previous_n_day(
        i + days_ago) for i in range(31)]

    corporate_restaurant_sales_last_month = get_all_corporate_restaurant_sales_as_pd(
        dates_in_last_months)

    re_open_counter = 0
    day_counter = 0

    while (re_open_counter < 8 and day_counter < 30):
        start_date = core.get_previous_n_day(day_counter + days_ago)
        if pd.to_datetime(start_date) in pd.to_datetime(corporate_restaurant_sales_last_month['day'].values):
            re_open_counter += 1
        day_counter += 1

    re_restaurant_list = get_corporate_restaurant_names()
    product_sold_for_re_dict = {}
    products_sold_for_re_pd = get_products_sold_for_re_as_pd(
        start_date, end_date)

    for restaurant_name in re_restaurant_list:
        product_sold_for_re_dict[restaurant_name] = products_sold_for_re_pd[
            products_sold_for_re_pd['shop_name'] == restaurant_name]

    all_stock_out_product_md = ""

    for restaurant_name in re_restaurant_list:
        products_sold_for_re = product_sold_for_re_dict[restaurant_name]
        sales_frequency = products_sold_for_re['product_name'].value_counts()

        for product, count in sales_frequency.items():
            if (count == 7):
                sales_today = products_sold_for_re.loc[(products_sold_for_re['product_name'] == product) & (
                    pd.to_datetime(products_sold_for_re['day']) == pd.to_datetime(today))]
                if (len(sales_today) == 0):
                    stock_out_product_details = get_stock_out_product_details_md(
                        product, restaurant_name)
                    all_stock_out_product_md += stock_out_product_details

    all_stock_out_product_html = markdown.markdown(all_stock_out_product_md)
    return all_stock_out_product_md, all_stock_out_product_html


In [ ]:
# | export

def get_cus_at_risk_of_loosing_title():
    cus_at_risk_of_loosing_title_md = get_cus_at_risk_of_loosing_title_md()
    cus_at_risk_of_loosing_title_md_html = markdown.markdown(
        cus_at_risk_of_loosing_title_md)

    return cus_at_risk_of_loosing_title_md, cus_at_risk_of_loosing_title_md_html


def get_cus_at_risk_of_loosing_list():
    today = get_today()
    eleven_weeks_ago = core.get_previous_n_day(11*7 + days_ago)
    pos_order_list = get_pos_order_as_pd(eleven_weeks_ago, today)

    pos_order_grouped = pos_order_list.groupby('customer_id')

    pos_order_grouped_by_cus_id = {}

    for cus_id, group_df in pos_order_grouped:
        pos_order_grouped_by_cus_id[cus_id] = group_df

    at_risk_of_loosing_title_cus_details_list_md = ""

    for cus_id, value in pos_order_grouped_by_cus_id.items():
        start_of_week = core.get_previous_n_day(1*7 + days_ago - 1)
        order_this_week = value[(pd.to_datetime(value['day']) >= pd.to_datetime(
            start_of_week)) & (pd.to_datetime(value['day']) <= pd.to_datetime(today))]
        this_week_order_count = len(order_this_week)

        if (this_week_order_count == 0):
            count = 0

            for i in range(1, 11):
                start_date = core.get_previous_n_day(
                    (i+1)*7 + days_ago - 1)
                end_date = core.get_previous_n_day(i*7 + days_ago)
                weekly_order = value[(pd.to_datetime(value['day']) >= pd.to_datetime(
                    start_date)) & (pd.to_datetime(value['day']) <= pd.to_datetime(end_date))]

                if (len(weekly_order) > 0):
                    count += 1

                if (count == 10):
                    total_purchase = value['amount_total'].sum()
                    customer_name = value.iloc[0, value.columns.get_loc(
                        'customer_name')]

                    grouped_by_location_df = value.groupby('location')
                    purchase_by_shop = {}
                    for location, group_df in grouped_by_location_df:
                        purchase_by_shop[location] = group_df['amount_total'].sum(
                        )
                    max_purchase_location = max(
                        purchase_by_shop, key=lambda k: purchase_by_shop[k]) if purchase_by_shop else 'undefined'

                    at_risk_of_loosing_title_cus_details_list_md += get_at_risk_of_loosing_title_cus_details_md(
                        customer_name, total_purchase, max_purchase_location)

    at_risk_of_loosing_title_cus_details_list_html = markdown.markdown(
            at_risk_of_loosing_title_cus_details_list_md)

    return at_risk_of_loosing_title_cus_details_list_md, at_risk_of_loosing_title_cus_details_list_html


In [ ]:
# | export

def get_daily_report():
    daily_report_md = ""
    daily_report_html = ""

    daily_report_top_md, daily_report_top_html = get_daily_report_top()
    daily_report_md += daily_report_top_md
    daily_report_html += daily_report_top_html

    store_sales_title_md, store_sales_title_html = get_store_sales_title()
    daily_report_md += store_sales_title_md
    daily_report_html += store_sales_title_html

    sales_per_shop_md, sales_per_shop_html = get_sales_per_shop()
    daily_report_md += sales_per_shop_md
    daily_report_html += sales_per_shop_html

    cus_at_risk_of_loosing_title_md, cus_at_risk_of_loosing_title_md_html = get_cus_at_risk_of_loosing_title()
    daily_report_md += cus_at_risk_of_loosing_title_md
    daily_report_html += cus_at_risk_of_loosing_title_md_html

    at_risk_of_loosing_title_cus_details_list_md, at_risk_of_loosing_title_cus_details_list_html = get_cus_at_risk_of_loosing_list()
    daily_report_md += at_risk_of_loosing_title_cus_details_list_md
    daily_report_html += at_risk_of_loosing_title_cus_details_list_html

    shop_product_stock_out_title_md, shop_product_stock_out_title_html = get_shop_product_stock_out_title()
    daily_report_md += shop_product_stock_out_title_md
    daily_report_html += shop_product_stock_out_title_html

    all_stock_out_product_md, all_stock_out_product_html = get_shop_product_likely_to_stock_out()
    daily_report_md += all_stock_out_product_md
    daily_report_html += all_stock_out_product_html

    re_sales_title_md, re_sales_title_html = get_re_sales_title()
    daily_report_md += re_sales_title_md
    daily_report_html += re_sales_title_html

    all_re_sales_md, all_re_sales_html = get_sales_per_re()
    daily_report_md += all_re_sales_md
    daily_report_html += all_re_sales_html

    shop_product_stock_out_title_md, shop_product_stock_out_title_html = get_re_product_stock_out_title()
    daily_report_md += shop_product_stock_out_title_md
    daily_report_html += shop_product_stock_out_title_html

    all_stock_out_product_md, all_stock_out_product_html = get_re_product_likely_to_stock_out()
    daily_report_md += all_stock_out_product_md
    daily_report_html += all_stock_out_product_html

    return daily_report_md, daily_report_html


In [ ]:
# | export

daily_report_md, daily_report_html = get_daily_report()
display_markdown(daily_report_md, raw=True)

# core.send_mail(to='yann@fthek.be', subject=get_email_subject(get_today()),
#           body=daily_report_html)
core.send_mail(to='niloy.shown+a1@gmail.com',
          subject=get_email_subject(get_today()), body=daily_report_html)


config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that
config.ini file available... using that



# 2023-04-25's - Tuesday -report

  

Today sales were 40775 EUR across the business:

  

- 0 EUR sold in stores

- 0 EUR sold in corporate restaurants

- 40775 EUR sold in events


## Store sales

### Stores sales per shop



- La Hulpe sold 0 EUR. Average of last 4 Tuesday’s was 1881 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- Fort Jaco sold 0 EUR. Average of last 4 Tuesday’s was 3087 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- Woluwe sold 0 EUR. Average of last 4 Tuesday’s was 3852 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- Tongres sold 0 EUR. Average of last 4 Tuesday’s was 3248 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- Wemmel sold 0 EUR. Average of last 4 Tuesday’s was 1821 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- CHATELAIN sold 0 EUR. Average of last 4 Tuesday’s was 3298 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- OVERIJSE sold 0 EUR. Average of last 4 Tuesday’s was 1940 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


- LATEM sold 0 EUR. Average of last 4 Tuesday’s was 2053 EUR. That’s **100.0%** decrease vs. average of last 4 Tuesday's


### Repeat customers we are at risk of loosing

Customers who didn’t buy in the last 7 days but who bought at least once a week for the last 10 weeks before that:

- NE CONSULT. Bought 402 EUR in the last 10 weeks. Buys the most in La Hulpe


- FLISCH GILLES. Bought 514 EUR in the last 10 weeks. Buys the most in Woluwe


- CAPITANI MAGALI. Bought 836 EUR in the last 10 weeks. Buys the most in Wemmel


- JANSSEN Thierry. Bought 342 EUR in the last 10 weeks. Buys the most in CHATELAIN


- STRONCIU Stefan. Bought 355 EUR in the last 10 weeks. Buys the most in CHATELAIN


### Products that were likely in stock out

The products that were sold in the each days of last 7 days for a given shop but didn’t sell today:
 

- FENOUIL GRILLE / OIGNONS / LAURIER / CORIANDRE / GRAINES DE MOUTARDE in La Hulpe


- VENTE COMPTOIR in La Hulpe


- LOTTE EPICEE in Woluwe


- SALADE COLESLAW : CHOU BLANC / CAROTTES / JEUNES OIGNONS / YAOURT (1 kg) in Woluwe


- BROCHETTE POISSON EPICEE in Woluwe


- LOTTE EPICEE in Wemmel


- SOUFFLE AU POIVRE NOIR 100G in CHATELAIN


- POULET DE FERME A LA BROCHE / ROMARIN  in CHATELAIN


- LOTTE EPICEE in CHATELAIN


- PAIN DE VIANDE AUX HERBES / TOMATES SECHEES (1 kg) in CHATELAIN


- HOMEMADE COOKIE FTK in CHATELAIN


- CHOU-FLEUR GRILLE / CROUTE D'EPICES / MOUTARDE / CURCUMA in CHATELAIN


- SESAMISO CHICKEN TENDERS (1 kg) in OVERIJSE


- PUREE DE POMMES DE TERRE NATURE 400G in LATEM


## Corporate restaurant sales

### Sales per corporate restaurant


There is no sales in the corporate restaurant today


### Products that were likely in stock out

The products that were sold in the each days of last 7 days for a given corporate restaurant but didn’t sell today:
 

There is no sales in the corporate restaurant today



In [ ]:
# core.send_mail(to='niloy.shown+a1@gmail.com',
#           subject=get_email_subject(get_today()), body=daily_report_html)

# core.send_mail(to='yann@fthek.be', subject=get_email_subject(get_today()),
#           body=daily_report_html)